# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,22.45,65,12,5.67,EC,1730922684
1,1,bethel,41.3712,-73.4140,26.48,56,0,7.20,US,1730922685
2,2,papatowai,-46.5619,169.4708,14.22,73,100,0.55,NZ,1730922686
3,3,olonkinbyen,70.9221,-8.7187,-2.39,82,14,20.96,SJ,1730922688
4,4,puquio,-14.7000,-74.1333,16.69,34,20,7.74,PE,1730922689


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# I'd like to go somewhere with highs between 55 and 75 F (roughly 13-23 C)
filtered_city_data = city_data_df.loc[(city_data_df['Max Temp']<=23) & (city_data_df['Max Temp']>=13)]

#Fixing Namibia country code
filtered_city_data.loc[filtered_city_data['Country'].isna(), 'Country'] = 'NA'

# Drop any rows with null values
np.where(pd.isnull(filtered_city_data))
np.where(pd.isna(filtered_city_data))
    #We have no missing or null values


# Display sample data
filtered_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,22.45,65,12,5.67,EC,1730922684
2,2,papatowai,-46.5619,169.4708,14.22,73,100,0.55,NZ,1730922686
4,4,puquio,-14.7000,-74.1333,16.69,34,20,7.74,PE,1730922689
6,6,port elizabeth,-33.9180,25.5701,13.51,87,40,1.03,ZA,1730922691
10,10,chinde,26.8212,87.7095,18.91,68,0,3.07,NP,1730922695


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,65,
2,papatowai,NZ,-46.5619,169.4708,73,
4,puquio,PE,-14.7000,-74.1333,34,
6,port elizabeth,ZA,-33.9180,25.5701,87,
10,chinde,NP,26.8212,87.7095,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 1
radius = 5000 #hotels within 5km of the coordinates
params = {
    "limit":limit,
    "categories": f"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
papatowai - nearest hotel: No hotel found
puquio - nearest hotel: Hostal Josuef
port elizabeth - nearest hotel: Waterford Hotel
chinde - nearest hotel: No hotel found
sanctuary point - nearest hotel: Dungowan Accommodation
jamestown - nearest hotel: DoubleTree Jamestown
adamstown - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
lebu - nearest hotel: No hotel found
k'olito - nearest hotel: ሀላባ ቪው አልጋ
hermanus - nearest hotel: Aloe guest house
lompoc - nearest hotel: Red Roof Inn Lompoc
shimoda - nearest hotel: ホテルウラガ
trinity beach - nearest hotel: Blue Lagoon Resort
ayora - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
tshikapa - nearest hotel: Hôtel TSHAMUNTU
port lincoln - nearest hotel: Boston Hotel
broken hill - nearest hotel: Ibis Styles
forster - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
ribeira grande 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,65,Hostal La Mirada De Solitario George
2,papatowai,NZ,-46.5619,169.4708,73,No hotel found
4,puquio,PE,-14.7000,-74.1333,34,Hostal Josuef
6,port elizabeth,ZA,-33.9180,25.5701,87,Waterford Hotel
10,chinde,NP,26.8212,87.7095,68,No hotel found
...,...,...,...,...,...,...
552,tamanrasset,DZ,22.7850,5.5228,22,فندق الأمان
556,bageshwar,IN,29.8500,79.7667,82,vivek hotel
569,el alto,PE,-4.2714,-81.2147,74,No hotel found
571,rantoul,US,40.3084,-88.1559,85,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)